In [3]:
import subprocess
import sys
import warnings
import os

# 1. Silent install
def silent_install(package):
    print(f"🔧 Installing {package}...")
    subprocess.run([sys.executable, "-m", "pip", "install", package],
                   stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    print(f"✅ Installed {package}")

for pkg in ["nltk", "gradio", "sentence-transformers", "huggingface_hub[hf_xet]" ]:
    silent_install(pkg)

# 2. Turn off FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# 3. Import NLTK
print("📦 Importing nltk and downloading stopwords...")
import nltk
nltk.download('stopwords', quiet=True)
print("✅ Stopwords downloaded")

# 4. Import core libraries
print("📚 Importing core libraries...")
import pandas as pd
import numpy as np
from collections import Counter
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import pickle
print("✅ Core libraries imported")


🔧 Installing nltk...
✅ Installed nltk
🔧 Installing gradio...
✅ Installed gradio
🔧 Installing sentence-transformers...
✅ Installed sentence-transformers
🔧 Installing huggingface_hub[hf_xet]...
✅ Installed huggingface_hub[hf_xet]
📦 Importing nltk and downloading stopwords...
✅ Stopwords downloaded
📚 Importing core libraries...
✅ Core libraries imported


In [5]:
# 5. Download of file from local storage (will be changed on kaggle storage)
def load_local_dataset(file_path: str):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"❌ File not found: {file_path}")

    if file_path.endswith(".xlsx"):
        df = pd.read_excel(file_path)
    elif file_path.endswith(".csv"):
        df = pd.read_csv(file_path)
    else:
        raise ValueError("❌ Supported only .csv и .xlsx")

    print("✅ Dataset downloaded from local storage.")
    return df

# Dataframe from file
df = load_local_dataset("/Users/anastasia/Desktop/DS_project/stephen-king-nlp/notebooks/data/primary.xlsx")


✅ Датасет загружен из локального файла.


In [7]:
# Recommendation function for similar books 

from sentence_transformers import SentenceTransformer

# Semantic analysis

# Uploading semantic model модель
model = SentenceTransformer('all-mpnet-base-v2')

texts = df["Wiki_Plot"].fillna("").tolist()
batch_size = 16
embeddings = []

for i in tqdm(range(0, len(texts), batch_size), desc="Encoding batches"):
    batch = texts[i:i+batch_size]
    emb = model.encode(batch)
    embeddings.extend(emb)

with open("plot_vectors.pkl", "wb") as f:
    pickle.dump(embeddings, f)

# Vectorization:
plot_embeddings = embeddings  

# Transform plot_embeddings into massive
plot_embeddings = np.array(plot_embeddings)

# Calculate similarity matrix
similarity_matrix = cosine_similarity(plot_embeddings)

# Get intex top-3 similar books
top3_indices = np.argsort(-similarity_matrix, axis=1)[:, 1:4]  # [1:4] — пропускаем саму себя

# Add names and covers into DataFrame
df["Top3_Titles"] = [[df.iloc[i]["Book_Title"] for i in row] for row in top3_indices]
df["Top3_Covers"] = [[df.iloc[i]["Cover_URL"] for i in row] for row in top3_indices]


Encoding batches: 100%|███████████████████████████| 6/6 [01:22<00:00, 13.81s/it]


In [23]:
# Preparing of short summary for search
import nltk
nltk.download('punkt_tab')

from nltk.tokenize import sent_tokenize
from transformers import pipeline
from tqdm import tqdm

# Uploading BART-model
summarizer_bart = pipeline("summarization", model="facebook/bart-large-cnn", tokenizer="facebook/bart-large-cnn")


[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/anastasia/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
Device set to use cpu


In [25]:
# Processing of plot summary for search

def split_text_by_sentences(text, max_words=500):
    sentences = sent_tokenize(text)
    chunks = []
    current_chunk = []
    word_count = 0

    for sent in sentences:
        sent_words = sent.split()
        if word_count + len(sent_words) > max_words:
            if current_chunk:
                chunks.append(" ".join(current_chunk))
            current_chunk = []
            word_count = 0
        current_chunk.extend(sent_words)
        word_count += len(sent_words)

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks


def summarize_blocks(chunks, summarizer, max_length=40, min_length=15):
    summaries = []
    for chunk in chunks:
        try:
            result = summarizer(chunk, max_length=max_length, min_length=min_length, do_sample=False)
            if result and 'summary_text' in result[0]:
                summaries.append(result[0]['summary_text'])
        except Exception as e:
            summaries.append(f"[Error: {e}]")
    return summaries


def summarize_wiki_plot_with_title(row):
    title = row["Book_Title"]
    text = row["Wiki_Plot"]

    print(f"🔄 Processing: {title}")

    try:
        chunks = split_text_by_sentences(text, max_words=500)
        partial_summaries = summarize_blocks(chunks, summarizer_bart)
        return " ".join(partial_summaries)
    except Exception as e:
        print(f"⚠️ Error during processing: {title}: {e}")
        return f"Summary error: {e}"




In [27]:
tqdm.pandas()
df["Short_Summary"] = df.fillna("").astype(str).progress_apply(summarize_wiki_plot_with_title, axis=1)

  0%|                                                    | 0/93 [00:00<?, ?it/s]

🔄 Обрабатывается: Carrie


  2%|▉                                           | 2/93 [00:35<27:12, 17.94s/it]

🔄 Обрабатывается: 'Salem's Lot


  3%|█▍                                          | 3/93 [00:57<28:56, 19.30s/it]

...

 99%|██████████████████████████████████████████▌| 92/93 [35:07<00:22, 22.74s/it]

🔄 Обрабатывается: You Like It Darker


100%|███████████████████████████████████████████| 93/93 [35:18<00:00, 19.38s/it]

🔄 Обрабатывается: Never Flinch


100%|███████████████████████████████████████████| 93/93 [35:26<00:00, 22.86s/it]


In [29]:
# Query for book search 
def search_books_by_query(query, top_n=5):
    query_vec = model.encode([query])
    similarities = cosine_similarity(query_vec, plot_embeddings)[0]

    df["Query_Similarity"] = similarities
    return df.sort_values("Query_Similarity", ascending=False)[["Book_Title", "Query_Similarity", "Short_Summary"]].head(top_n)


In [43]:
# Check search quality
search_books_by_query("Deschain", top_n=5)

,Book_Title,Query_Similarity,Short_Summary
70,The Dark Tower: The Wind Through the Keyhole,0.346064,The novel begins with Roland and his ka-tet ar...
24,The Eyes of the Dragon,0.305314,"King Roland's magician, Flagg, seeks to destro..."
35,Nightmares & Dreamscapes,0.282406,Nightmares & Dreamscapes is another virtuoso ...
56,The Dark Tower VII: The Dark Tower,0.277475,"Mia, her body now physically separated from Su..."
25,The Dark Tower II: The Drawing of the Three,0.242158,Roland wakes up on a beach and is attacked by ...


In [93]:
import gradio as gr
import io
from contextlib import redirect_stdout
import re


# Search function
def search_interface(query):
    results = search_books_by_query(query, top_n=5)
    formatted = "\n\n".join([
        f"📖 {row['Book_Title']}\n📝 {row['Short_Summary']}"
        for _, row in results.iterrows()
    ])
    return formatted

# Gradio interface
demo = gr.Interface(
    fn=search_interface,
    inputs=gr.Textbox(label="🔍 Search for a theme or plot element"),
    outputs=gr.Textbox(label="📚 Matching books and summaries"),
)

# Перехватываем вывод, чтобы извлечь ссылку
f = io.StringIO()
with redirect_stdout(f):
    demo.launch(share=True, prevent_thread_lock=True)

output = f.getvalue()
match = re.search(r'(https://[a-z0-9\-]+\.gradio\.live)', output)
gradio_url = match.group(1) if match else None


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [95]:
print("📎 Автоматически извлечённый URL:", gradio_url)

📎 Автоматически извлечённый URL: https://27a470d30fae5e5e6e.gradio.live


In [101]:
# HTML-page with dashboard and search

df_sorted = df.sort_values(by="Year")

html_blocks = ""
for i, (_, row) in enumerate(df_sorted.iterrows()):
    rating = row['goodreads_rating']
    percentage = (rating / 5) * 100
    book_title = row['Book_Title']
    cover_url = row['Cover_URL']
    year = int(row['Year'])

    # Position class
    position_class = ""
    if i == 0:
        position_class = "first-item"
    elif i == len(df_sorted) - 1:
        position_class = "last-item"

    # Similar books generation
    similar_html = ""
    if isinstance(row["Top3_Covers"], list) and isinstance(row["Top3_Titles"], list):
        for sim_cover, sim_title in zip(row["Top3_Covers"], row["Top3_Titles"]):
            similar_html += f"""
            <div class="similar-book">
                <img src="{sim_cover}" class="similar-cover" alt="{sim_title}">
                <div class="similar-title">{sim_title}</div>
            </div>
            """

    # HTML for timeline
    html_blocks += f"""
    <div class="book-item {position_class}">
        <div class="book-cover-wrapper">
            <div class="star-rating">
                <div class="stars-outer">
                    <div class="stars-inner" style="width: {percentage:.2f}%;"></div>
                </div>
            </div>
            <div class="tooltip">
                <div class="cover-container">
                    <img src="{cover_url}" class="book-cover">
                </div>
                <div class="similar-popup">
                    <div class="similar-label">Recommended</div>
                    <div class="similar-container">
                        {similar_html}
                    </div>
                </div>
            </div>
            <div class="book-title">{book_title}</div>
            <div class="book-year">{year}</div>
        </div>
    </div>
    """




# Styles
timeline_html = f"""
<style>
    html, body {{
        height: auto;
        overflow-x: hidden;
        overflow-y: auto;
        margin: 0;
        padding: 0;
    }}
    .timeline-container {{
        display: flex;
        flex-wrap: nowrap;
        overflow-x: auto;
        gap: 20px;
        padding: 20px 0;
        height: 600px;
        
    }}
    .book-item {{
        display: flex;
        flex: 0 0 auto;
        flex-direction: column;
        text-align: center;
        align-items: center;
        max-width: 160px;
        position: relative;
        font-family: sans-serif;
    }}
    .book-item.first-item .similar-popup {{
        left: 0;
        transform: none;
    }}
    .book-item.last-item .similar-popup {{
        right: 0;
        left: auto;
        transform: none;
    }}
    .book-cover {{
        width: 100%;
        border-radius: 8px;
        box-shadow: 0 4px 6px rgba(0,0,0,0.3);
        transition: transform 0.3s ease;
    }}
    .cover-container {{
        height: 290px; 
        display: flex;
        align-items: flex-start; /* sam level for top */
        justify-content: center;
    }}
    .book-cover:hover {{
        transform: scale(1.2);
        z-index: 2;
    }}
    .tooltip {{
        position: relative;
        display: inline-block;
    }}
    .tooltiptext {{
        visibility: hidden;
        width: 180px;
        background-color: #222;
        color: #fff;
        text-align: left;
        border-radius: 6px;
        padding: 8px;
        position: absolute;
        z-index: 10;
        bottom: 110%;
        left: 50%;
        transform: translateX(-50%);
        opacity: 0;
        transition: opacity 0.3s;
        font-size: 12px;
    }}
    .tooltip:hover .tooltiptext {{
        visibility: visible;
        opacity: 1;
    }}
    .book-title {{
        font-weight: bold;
        margin-top: 5px;
    }}
    .book-year, .book-genre {{
        font-size: 12px;
        color: #666;
    }}

   .star-rating {{
        display: block;
        text-align: center;
        margin-bottom: 20px;
        font-size: 14px;
        position: relative;
        unicode-bidi: bidi-override;
    }}

    .stars-outer {{
        color: #ccc;
        position: relative;
        display: inline-block;
        font-size: 18px;
    }}
    .stars-outer::before {{
        content: "★★★★★";
    }}
    .stars-inner {{
        color: #f39c12;
        position: absolute;
        top: 0;
        left: 0;
        white-space: nowrap;
        overflow: hidden;
        width: 0;
    }}
    .stars-inner::before {{
        content: "★★★★★";
    }}
    .similar-popup {{
        display: none;
        position: absolute;
        bottom: -210px;
        left: 50%;
        transform: translateX(-50%);
        background-color: #fff;
        border: 1px solid #ddd;
        padding: 10px;
        box-shadow: 0 4px 10px rgba(0,0,0,0.1);
        z-index: 5;
        border-radius: 8px;
        text-align: center;
        flex-direction: column;
        align-items: center;
        width: auto;
    }}
    .tooltip:hover .similar-popup {{
        display: flex;
        justify-content: center;
        gap: 10px;
    }}
    .similar-book {{
        display: flex;
        flex-direction: column;
        align-items: center;
        width: 90px;
    }}
    .similar-cover {{
        width: 100%;
        border-radius: 4px;
        box-shadow: 0 2px 4px rgba(0,0,0,0.2);
    }}
    .similar-title {{
        font-size: 10px;
        margin-top: 5px;
        color: #333;
    }}
    .similar-label {{
        font-weight: bold;
        font-size: 12px;
        margin-bottom: 8px;
        color: #222;
        width: 100%;
    }}
    .similar-container {{
        display: flex;
        justify-content: center;
        gap: 10px;
    }}
</style>
<div class="timeline-container">
    {html_blocks}
</div>
"""
timeline_html += f"""
<div style="margin-top: 60px;">
  <h2 style="text-align:center;">🔍 Stephen King Book Finder </h2>
  <iframe 
    src="{gradio_url}" 
    width="100%" 
    height="650" 
    style="border:none; overflow:hidden; border-radius: 12px;">
  </iframe>
</div>
"""


# Show
from IPython.display import display, HTML
display(HTML(timeline_html))

In [105]:
output_path = "/Users/anastasia/Desktop/DS_project/stephen-king-nlp/notebooks/data/index.html"

with open(output_path, "w", encoding="utf-8") as f:
    f.write(timeline_html)

print("✅ Сохранено в:", output_path)

✅ Сохранено в: /Users/anastasia/Desktop/DS_project/stephen-king-nlp/notebooks/data/index.html
